## AutoGPT 智能体

以下`SERPAPI_API_KEY`仅为示例，请访问 https://serpapi.com 注册账号并替换为自己的 `API_KEY`（每月100次免费调用）

In [4]:
import os

os.environ["SERPAPI_API_KEY"] = "1e3c93e0753ac224098370cd71da86150c6609caba6b5aaa307e04b72a5006e1"

In [5]:
from langchain.utilities import SerpAPIWrapper
from langchain.agents import Tool
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool


In [6]:
# 构造 AutoGPT 的工具集
search = SerpAPIWrapper()
tools = [
    Tool(
        name="search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions",
    ),
    WriteFileTool(),
    ReadFileTool(),
]

In [7]:
from langchain_openai import OpenAIEmbeddings

In [8]:
# OpenAI Embedding 模型
embeddings_model = OpenAIEmbeddings()

In [9]:
import faiss
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore

# OpenAI Embedding 向量维数
embedding_size = 1536
# 使用 Faiss 的 IndexFlatL2 索引
index = faiss.IndexFlatL2(embedding_size)
# 实例化 Faiss 向量数据库
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


## 实例化自主智能体 Auto-GPT

In [10]:
from langchain_experimental.autonomous_agents import AutoGPT
from langchain_openai import ChatOpenAI

In [11]:
agent = AutoGPT.from_llm_and_tools(
    ai_name="Jarvis",
    ai_role="Assistant",
    tools=tools,
    llm=ChatOpenAI(model_name="gpt-4-", temperature=0, verbose=True),
    memory=vectorstore.as_retriever(
        search_type="similarity_score_threshold",
        search_kwargs={"score_threshold": 0.8}),# 实例化 Faiss 的 VectorStoreRetriever
)

In [12]:
# 打印 Auto-GPT 内部的 chain 日志
agent.chain.verbose = True

In [13]:
agent.run(["2023年成都大运会，中国金牌数是多少"])



> Entering new LLMChain chain...


/root/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain/schema/vectorstore.py:325: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


Prompt after formatting:
System: You are Jarvis, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. 2023年成都大运会，中国金牌数是多少


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: useful for when you need to answer questions about current events. You should ask targeted questions, args json schema: {"tool_input": {"type": "string"}}
2. write_file: Write file to disk, args json schema: {"file_path": {"title": "File Path", "description

'I have found the number of gold medals China won in the 2023 Chengdu Universiade and saved this information for future reference. I have now completed all the objectives.'